# Python Resource Management

# Documentations

* [PEP 343 – The “with” Statement](https://peps.python.org/pep-0343/)

* [Context Manager Types](https://docs.python.org/3/library/stdtypes.html#context-manager-types)

> Python’s [with](https://docs.python.org/3/reference/compound_stmts.html#with) statement is implemented using **a pair of methods** that allow user-defined classes to define a runtime context
>
> * contextmanager.__enter__()  
> Enter the runtime context and return either this object or another object related to the runtime context.  
> 
> * contextmanager.__exit__(exc_type, exc_val, exc_tb)  
> If an exception occurred while executing the body of the with statement, Python runtime passes the arguments contain the exception type, value and traceback information. Otherwise, all three arguments are ```None```. Returning a ```True``` from this method will cause the with statement to suppress the exception and continue execution with the statement immediately following the with statement. Otherwise the exception continues propagating after this method has finished executing.
>
> Python’s generators and the contextlib.contextmanager decorator provide a convenient way to implement these protocols. If a generator function is decorated with the contextlib.contextmanager decorator, it will return a context manager implementing the necessary __enter__() and __exit__() methods, rather than the iterator produced by an undecorated generator function.

* [@contextlib.contextmanager](https://docs.python.org/3/library/contextlib.html#contextlib.contextmanager)

> define a factory function for with statement context managers, without needing to create a class or separate __enter__() and __exit__() methods.
> ```
> from contextlib import contextmanager
> 
> @contextmanager
> def managed_resource(*args, **kwds):
>     # Code to acquire resource, e.g.:
>     resource = acquire_resource(*args, **kwds)
>     try:
>         yield resource
>     finally:
>         # Code to release resource, e.g.:
>         release_resource(resource)
> ```

* [Python Tips - 27. Context Managers](https://book.pythontips.com/en/latest/context_managers.html#implementing-a-context-manager-as-a-class)

> ```
> class File(object):
>     def __init__(self, file_name, method):
>         self.file_obj = open(file_name, method)
>     def __enter__(self):
>         return self.file_obj
>     def __exit__(self, type, value, traceback):
>         print("Exception has been handled")
>         self.file_obj.close()
>         return True
> ```
>
> Implementing a Context Manager as a Generator  
> ```
> from contextlib import contextmanager
> 
> @contextmanager
> def open_file(name):
>     f = open(name, 'w')
>     try:
>         yield f
>     finally:
>         f.close()
> ```

[Context Managers and Python's with Statement](https://realpython.com/python-with-statement/)

* What the Python with statement is for and how to use it
* What the context management protocol is
* How to implement your own context managers

Two general approaches to deal with resource lifecycle management. 

1. A try … finally construct
2. A with construct

The context manager object results from evaluating the expression after with. In other words, expression must return an object that implements the **context management protocol. This protocol consists of two special methods**:

* __enter__() is called by the with statement to enter the runtime context.
* __exit__() is called when the execution leaves the with code block.

You can provide the same functionality by implementing both the .__enter__() and the .__exit__() special methods in your class-based context managers. You can also create custom function-based context managers using the contextlib.contextmanager decorator from the standard library and an appropriately coded generator function.

# Class destructor

* [How do I correctly clean up a Python object?](https://stackoverflow.com/a/865272/4281353)

> To use the ```with``` statement, create a class with the following methods:
> ```
> def __enter__(self)
> def __exit__(self, exc_type, exc_value, traceback)
> ```
```
class Package:
    def __init__(self):
        self.files = []

    def __enter__(self):
        return self

    # ...

    def __exit__(self, exc_type, exc_value, traceback):
        for file in self.files:
            os.unlink(file)


with Package() as package_obj:
    # use package_obj
```

> The standard way is to use [atexit.register](https://docs.python.org/3/library/atexit.html#atexit.register):
```
# package.py
import atexit
import os

class Package:
    def __init__(self):
        self.files = []
        atexit.register(self.cleanup)

    def cleanup(self):
        print("Running cleanup...")
        for file in self.files:
            print("Unlinking file: {}".format(file))
            # os.unlink(file)
```

# Multiple Resources using ExitStack

New in Python 3.10.

* [contextlib.ExitStack)](https://docs.python.org/3/library/contextlib.html#contextlib.ExitStack)

> ```
> with ExitStack() as stack:
>    files = [stack.enter_context(open(fname)) for fname in filenames]
> ```    